In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data= pd.read_csv('/content/adm_data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [3]:
columnas_eliminar = ['Serial No.']
data = data.drop(columnas_eliminar, axis=1)

In [4]:
data.to_csv("nuevodata.csv",index=False)

In [5]:
data = np.loadtxt("/content/nuevodata.csv", delimiter=',',skiprows=1)
# print(data)
X_data = data[:, :7]
Y_data = data[:, 7]
print(X_data.shape)
print(Y_data.shape)

(400, 7)
(400,)


In [6]:
#X_train=X[:13000]
#X_test=X[13000:]
# Calcular la media y la desviación estándar de cada característica en el conjunto de entrenamiento.
#mean = np.mean(X_train, axis=0)
#std = np.std(X_train, axis=0)
# Normalizar los datos de entrenamiento.
#X_train_normalized = (X_train - mean) / std
# Normalizar los datos de prueba utilizando la misma media y desviación estándar.
#X_test_normalized = (X_test - mean) / std

#y_train= Y[:13000]
#y_test= Y[13000:]


# Normalizar los datos
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

In [7]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

X_train_t = torch.from_numpy(X_train).float().cuda()
Y_train_t = torch.from_numpy(Y_train).float().cuda()
X_test_t = torch.from_numpy(X_test).float().cuda()

In [8]:
# Crear una clase personalizada para el modelo de regresión
class ModeloRegresion(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        super(ModeloRegresion, self).__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(H, D_out)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [9]:
# Definir dimensiones de entrada, unidades ocultas y salida
D_in, H, D_out = 7, 100, 1

# Crear una instancia del modelo
modelo = ModeloRegresion(D_in, H, D_out)

# Mover el modelo a la GPU si está disponible
modelo.to("cuda")

ModeloRegresion(
  (fc1): Linear(in_features=7, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

In [10]:
# Definir la función de pérdida y el optimizador

# Usar Mean Squared Error para regresión
criterio = nn.MSELoss() # nn.MSELoss(), que es la función de pérdida de error cuadrático medio
#criterio = nn.L1Loss() #error absoluto medio (MAE), debes cambiarla a nn.L1Loss()
optimizador = optim.SGD(modelo.parameters(), lr=0.001)

In [11]:
# Entrenamiento del modelo
epochs = 100
log_each = 10
l = []

modelo.train()
for e in range(1, epochs + 1):
    # Forward
    y_pred = modelo(X_train_t)
    #print(y_pred)
    # Calcular la pérdida
    loss = criterio(y_pred, Y_train_t)
    l.append(loss.item())

    # Reiniciar los gradientes
    optimizador.zero_grad()

    # Retropropagación
    loss.backward()

    # Actualizar los pesos
    optimizador.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([320])) that is different to the input size (torch.Size([320, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 10/100 Loss 0.33671
Epoch 20/100 Loss 0.29630
Epoch 30/100 Loss 0.26427
Epoch 40/100 Loss 0.23864
Epoch 50/100 Loss 0.21792
Epoch 60/100 Loss 0.20100
Epoch 70/100 Loss 0.18704
Epoch 80/100 Loss 0.17540
Epoch 90/100 Loss 0.16559
Epoch 100/100 Loss 0.15725


In [12]:
from sklearn.metrics import accuracy_score #libreria para calcular la precisión de las predicciones
from sklearn.metrics import mean_squared_error
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def evaluate(x):
    modelo.eval()#poner el modelo en modo evaluacion
    y_pred = modelo(x) # realiza una inferencia en el modelo usando el tensor de entrada X para obtenera las predicciones
    y_probas = softmax(y_pred)#para obtener la probabilidades de clase, Convierte las puntuaciones del modelo en distribuciones de porbabilidad
    return torch.argmax(y_probas, axis=1)#encontrar la clase con la probabilidad mas alta para cada modelo

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())#Realiza la evaluación del modelo en el conjunto de datos de prueba
mse = mean_squared_error(Y_test, y_pred.cpu().numpy())
print(f"Mean Squared Error (MSE): {mse:.5f}")

Mean Squared Error (MSE): 0.54512
